In [1]:
print('test')

test


In [ ]:
# corerenza di utilizzo nomi per etichette

#### label_id    ( etichetta codice, es:  "A03"  )
###  label_text  ( etichetta testuale, es:  "Colera"  )
###  source_text ( testo finale da embeddare: "Colera, malattia colerica ..."" )

## Labels

### Labels data cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


malat_Cat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='categorie_mlt_tmt',header=1)
malat_SubCat = pd.read_excel("../data_icd10/Codici-ICD10-categorie_mlt_tmt.xlsx", sheet_name='sottocategorie_mlt_tmt',header=1)

##############################
# Rename columns, but keep old names:
column_name_reference_CAT = {
    "Codice Categoria Malattie e Traumatismi": "Codice_categoria",
    "Capitolo": "Capitolo",
    "Blocchi di categorie": "Blocco",
    "Descrizione delle categorie di Malattie e traumatismi": "Descr_Categ"
}
# Apply renaming
malat_Cat = malat_Cat.rename(columns=column_name_reference_CAT)

column_name_reference_SubCat = {
    "Codice Categoria Malattie e traumatismi": "Codice_categoria",
    "Codice sottocategoria Malattie e Traumatismi": "Codice_SottoCategoria",
    "Descrizione delle sottocategorie di Malattie e traumatismi": "Descr_SottoCateg"
}

#
malat_SubCat = malat_SubCat.rename(columns=column_name_reference_SubCat)

malat_SubCat['Codice_SottoCategoria']= malat_SubCat['Codice_SottoCategoria'].astype('Int64')
malat_Cat['Capitolo'] = malat_Cat['Capitolo'].astype('Int64')
malat_Cat['Blocco'] = malat_Cat['Blocco'].astype('Int64')

# elimina righe vuote
malat_Cat = malat_Cat.dropna()
malat_SubCat = malat_SubCat.dropna()

display(malat_Cat.head(3))
print('-------------')
display(malat_SubCat.head(7))




,Codice_categoria,Capitolo,Blocco,Descr_Categ
0,A00,1,1,Colera
1,A01,1,1,Febbre tifoide e paratifo
2,A02,1,1,Altre infezioni da salmonella


-------------


,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A
5,A01,2,Paratifo B
6,A01,3,Paratifo C


In [2]:
# Goal is to concatenate all subcat text for each categ.
# Concatenate text in each category with groupby and join

df_subcat_text= malat_SubCat.groupby('Codice_categoria')['Descr_SottoCateg'].apply(lambda text: ' '.join(text))

# and also remake it a proper dataframe with reset index, 
# so that it can be merged back
df_subcat_text= df_subcat_text.reset_index()
df_subcat_text.columns = ['Codice_categoria', 'Descr_SottoCateg_Estesa']

display(malat_SubCat.head(5))
df_subcat_text.head(3)

,Codice_categoria,Codice_SottoCategoria,Descr_SottoCateg
0,A00,0,"Colera da Vibrio cholerae 01, biotipo del colera"
1,A00,1,"Colera da Vibrio cholerae 01, biotipo eltor"
2,A00,9,Colera non specificato
3,A01,0,Febbre tifoide
4,A01,1,Paratifo A


,Codice_categoria,Descr_SottoCateg_Estesa
0,A00,"Colera da Vibrio cholerae 01, biotipo del cole..."
1,A01,Febbre tifoide Paratifo A Paratifo B Paratifo ...
2,A02,Enterite da salmonella Setticemia da salmonell...


In [3]:
#### merge

malat_Cat_full = malat_Cat.merge(
    how='left',
    right= df_subcat_text,
    on= 'Codice_categoria',
)

# sostituisce NaN con stringa vuota per fare unione finale stringhe
malat_Cat_full['Descr_SottoCateg_Estesa'] =  malat_Cat_full['Descr_SottoCateg_Estesa'].fillna('')
malat_Cat_full.tail()

# crea descrizione totale unendo anche descrizione base della categoria 
malat_Cat_full['Descr_totale_Estesa'] =  malat_Cat_full['Descr_Categ'] + ': ' + malat_Cat_full['Descr_SottoCateg_Estesa']
malat_Cat_full.tail(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
1949,Y96,20,35,Condizione associata al lavoro,Condizione associata al lavoro,Condizione associata al lavoro: Condizione ass...
1950,Y97,20,35,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambienta...
1951,Y98,20,35,Condizione associata allo stile di vita,Condizione associata allo stile di vita,Condizione associata allo stile di vita: Condi...


#### Final Cleaned Dataframe to use for collection

In [4]:
malat_Cat_full.head()

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...
2,A02,1,1,Altre infezioni da salmonella,Enterite da salmonella Setticemia da salmonell...,Altre infezioni da salmonella: Enterite da sal...
3,A03,1,1,Shigellosi,Shigellosi da Shigella dysenteriae Shigellosi ...,Shigellosi: Shigellosi da Shigella dysenteriae...
4,A04,1,1,Altre infezioni batteriche intestinali,Infezione da Escherichia coli enteropatogena I...,Altre infezioni batteriche intestinali: Infezi...


### Labels create collection

In [5]:
# Dai puliti:
malat_Cat_full.tail(3)

,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
1949,Y96,20,35,Condizione associata al lavoro,Condizione associata al lavoro,Condizione associata al lavoro: Condizione ass...
1950,Y97,20,35,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambientale,Condizione associata all'inquinamento ambienta...
1951,Y98,20,35,Condizione associata allo stile di vita,Condizione associata allo stile di vita,Condizione associata allo stile di vita: Condi...


In [6]:
# per usare i moduli 
# da temrinal fare pip install -e . 
# ( "e"  sta per editing )
# cosi installa tutto e perfmettte di fare modifiche ai moduli e riavviare e riusarli



In [7]:
df_copy = malat_Cat_full.copy()
df_copy.head(2)


,Codice_categoria,Capitolo,Blocco,Descr_Categ,Descr_SottoCateg_Estesa,Descr_totale_Estesa
0,A00,1,1,Colera,"Colera da Vibrio cholerae 01, biotipo del cole...","Colera: Colera da Vibrio cholerae 01, biotipo ..."
1,A01,1,1,Febbre tifoide e paratifo,Febbre tifoide Paratifo A Paratifo B Paratifo ...,Febbre tifoide e paratifo: Febbre tifoide Para...


In [8]:
from semantic_search.data import Document, Corpus

corpus_df= Corpus.from_dataframe(
    df=df_copy,
    source_text_col='Descr_totale_Estesa',
    label_text_col='Descr_Categ',
    label_id_col='Codice_categoria',
    father_label_id_col='Blocco',
    g_father_label_id_col='Capitolo'
      )

corpus_df.documents[:5]

[Document(id=0, content='Colera: Colera da Vibrio cholerae 01, biotipo del colera Colera da Vibrio cholerae 01, biotipo eltor Colera non specificato', metadata={'label_id': 'A00', 'label_text': 'Colera', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=1, content='Febbre tifoide e paratifo: Febbre tifoide Paratifo A Paratifo B Paratifo C Paratifo non specificato', metadata={'label_id': 'A01', 'label_text': 'Febbre tifoide e paratifo', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=2, content='Altre infezioni da salmonella: Enterite da salmonella Setticemia da salmonella Infezioni localizzate da salmonella Altre infezioni da salmonella, specificate Infezione da salmonella, non specificata', metadata={'label_id': 'A02', 'label_text': 'Altre infezioni da salmonella', 'father_label_id': 1, 'g_father_label_id': 1}),
 Document(id=3, content='Shigellosi: Shigellosi da Shigella dysenteriae Shigellosi da Shigella flexneri Shigellosi da Shigella boydii Shigellosi da Shi

In [10]:
from semantic_search.qdrant import EmbeddingModel, CollectionMananger

cm = CollectionMananger()

cm.list_collections()

['istat-data-384',
 'test-new-script',
 'test-collection-dev',
 'istat-data-v2-3072',
 'istat-data-open',
 'test-collection-01',
 'istat-icd10_bis',
 'istat-data-3072',
 'istat-data',
 'istat-icd10',
 'istat-ateco',
 'dummy_object_labels',
 'istat-data-AP',
 'istat-icd10_Factory_0']

In [10]:
# cm.create(
#     name="istat-icd10_Factory_0",
#     corpus=corpus_df,
#     model_id="text-embedding-ada-002",
#     model_type="openai",
#     vector_size=1536,     
#     overwrite=True
# )

Uploading batches to istat-icd10_Factory_0: 100%|██████████| 61/61 [00:57<00:00,  1.07it/s]


In [ ]:
# cm.create(
#     name="istat-icd10_Factory_0",
#     corpus=corpus_df,
#     model_id="text-embedding-ada-002",
#     model_type="openai",
#     vector_size=1536,     
#     overwrite=True,
#     embed_batch_size= 128
# )

Uploading batches to istat-icd10_Factory_0: 100%|██████████| 61/61 [02:15<00:00,  2.23s/it]


In [11]:
cm.get_collection_info('istat-icd10_Factory_0')

{'name': 'istat-icd10_Factory_0',
 'status': 'green',
 'vector_size': 1536,
 'distance': 'Cosine',
 'points_count': 1952}

#### Test Seeking on collection

In [13]:
# Single Query seeking 

from semantic_search.qdrant import SemanticSeeker

seeker = SemanticSeeker(
    collection_name="istat-icd10_Factory_0",
    model_id="text-embedding-ada-002",
    model_type="openai",
    vector_size=1536,       # ada-002 will default to its fixed 1536
)

# 2) Run the search
top_findings = seeker.search(query='Scarlattina', top_k=3 )
top_findings.results



Creating embeddings...: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s]


[RetrievedPoint(id=32, metadata={'label_id': 'A38', 'label_text': 'Scarlattina', 'father_label_id': 4, 'g_father_label_id': 1, 'source_text': 'Scarlattina: Scarlattina'}, score=0.9711024),
 RetrievedPoint(id=159, metadata={'label_id': 'B86', 'label_text': 'Scabbia', 'father_label_id': 18, 'g_father_label_id': 1, 'source_text': 'Scabbia: Scabbia'}, score=0.8494627),
 RetrievedPoint(id=898, metadata={'label_id': 'L84', 'label_text': "Calli e callosita'", 'father_label_id': 8, 'g_father_label_id': 12, 'source_text': "Calli e callosita': Calli e callosita'"}, score=0.8409397)]

In [14]:
# Multiple Query seeking 
from semantic_search.qdrant import SemanticSeeker

seeker = SemanticSeeker(
    collection_name="istat-icd10_Factory_0",
    model_id="text-embedding-ada-002",
    model_type="openai",
    vector_size=1536,       # ada-002 will default to its fixed 1536
)

queries_list  = ["Scarlattina", "Epilessia", "Polmonite"]
results= seeker.search_many(queries= queries_list, top_k= 3)
results


Creating embeddings...: 100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


TypeError: RetrievedPoint.__init__() got an unexpected keyword argument 'text'

## Test-Data

### Test-Data Cleaning

In [56]:
# Dati Reali da testare:
df_note= pd.read_excel('../data_icd10/TRAINING_medcod_012022.xlsx')
df_note.head(4)
print(df_note.shape)

(253626, 7)


In [57]:
df_note['linecoded']= df_note['linecoded'].astype('Int64')
df_note['linenb']= df_note['linenb'].astype('Int64')
df_note.head(4)

,id,linenb,cod_causa,testo_originale,intervallo_causa,codeonly,linecoded
0,1,0,R579,COLLASSO CARDIOCIRCOLATORIO,NaN,NaN,1
1,1,2,C799(2Months),METASTASI,02M,NaN,1
2,1,3,C159(4Months),ADENOCARCINOMA ESOFAGEO,04M,NaN,1
3,2,0,R570,COLLASSO CARDIORESPIRATORIO ACUTO,NaN,NaN,1


In [58]:
# REMOVE any parenthesis and their contents, trim whitespace, and insert as a new column right after 'cod_causa'
cleaned = (
    df_note['cod_causa']
    .str.replace(r'\s*\([^)]*\)', '', regex=True)  # strip “ (… )” and any preceding spaces
    .str.strip()                                    # remove leading/trailing spaces
)

insert_loc = df_note.columns.get_loc('cod_causa') + 1
df_note.insert(insert_loc, 'cod_causa_clean', cleaned)

df_note.tail(4)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
253622,72346,1,R630(1Months); R64(1Months); N19(1Months),R630; R64; N19,ANORESSIA; CACHESSIA; INSUFFICIENZA RENALE,01M,NaN,1
253623,72346,2,C786(4Months),C786,CARCINOSI PERITONEALI ASCITOGENA,04M,NaN,1
253624,72346,3,C187(4Months),C187,ADENOCARCINOMA SIGMA,04M,NaN,1
253625,72346,5,I6399; G319(5Months)(SEQ); D735; D350; C439(5M...,I6399; G319; D735; D350; C439,ICTUS ISCHEMICO AORTICO SOTTOCORTICALE CON MOD...,NaN,NaN,1


In [59]:
# prendiamo solo righe con UN codice , NON codici multipli ( legati a piu malattie del paziente)

df_single = df_note[
    df_note['cod_causa_clean'].str.match(r'^[A-Za-z0-9]+$', na=False)
]
df_single.tail(4)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
253620,72345,2,J960(3Days),J960,INSUFFICIENZA RESPIRATORIA ACUTA,03G,NaN,1
253621,72345,3,J841(4Years),J841,FIBROSI POLMONARE,04A,NaN,1
253623,72346,2,C786(4Months),C786,CARCINOSI PERITONEALI ASCITOGENA,04M,NaN,1
253624,72346,3,C187(4Months),C187,ADENOCARCINOMA SIGMA,04M,NaN,1


In [60]:
# alcune rihce hanno codici non riconoscibili, piu di 4 caratteri, esempio:
df_single[ df_single['cod_causa_clean'].str.len() >4 ].head(3)

,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
18,7,0,I2199(1Days),I2199,INFARTO MIOCARDICO ACUTO,01G,NaN,1
38,14,2,I6409,I6409,ICTUS CEREBRALE,NaN,NaN,1
74,25,0,I6409(1Days),I6409,ICTUS CEREBRI,01G,NaN,1


In [62]:
# TOGLIAMO RIGHE con 'cod_causa_clean' con piu di 3 caratteri, che non nesistono per noi 

df_single= df_single[ df_single['cod_causa_clean'].str.len() <= 4 ]

In [63]:
# Ora ci sono sia codici da 4 che da 3 carateri ( categorie )
#
df_single[ df_single['cod_causa_clean'].str.len() <= 3 ].head(3)


,id,linenb,cod_causa,cod_causa_clean,testo_originale,intervallo_causa,codeonly,linecoded
12,5,3,R54(7Days),R54,CACHESSIA SENILE,07G,NaN,1
20,7,3,I10(13Years),I10,IPERTENSIONE ARTERIOSA,13A,NaN,1
24,8,3,I10(20Years),I10,IPERTENSIONE ARTERIOSA,20A,NaN,1


In [ ]:
#  Formiamo coonna "categoria_clean"  (la categoria ha 3 caratteri ) :
#  togliendo ultimo numero di 'cod_causa_clean' se sono 4 caratteri,
#  altrimenti lo lascia cosi perche 3 caratteri sono gia categoria 

# calcola la posizione subito dopo 'cod_causa_clean'
pos = df_single.columns.get_loc('cod_causa_clean') + 1

# crea la nuova serie: se la lunghezza è 4, togli l’ultimo carattere, altrimenti lasciala invariata
categoria = df_single['cod_causa_clean'].astype(str).apply(lambda x: x[:-1] if len(x) >= 4 else x)

# inserisci la colonna nella posizione desiderata
df_single.insert(pos, 'categoria_clean', categoria)


In [67]:
print(df_single.shape)
df_single.tail(4)

(186555, 9)


,id,linenb,cod_causa,cod_causa_clean,categoria_clean,testo_originale,intervallo_causa,codeonly,linecoded
253620,72345,2,J960(3Days),J960,J96,INSUFFICIENZA RESPIRATORIA ACUTA,03G,NaN,1
253621,72345,3,J841(4Years),J841,J84,FIBROSI POLMONARE,04A,NaN,1
253623,72346,2,C786(4Months),C786,C78,CARCINOSI PERITONEALI ASCITOGENA,04M,NaN,1
253624,72346,3,C187(4Months),C187,C18,ADENOCARCINOMA SIGMA,04M,NaN,1


### Test-Data Results

In [68]:
df_noteSample10 = df_single.sample(n=1000, random_state=10)
df_noteSample10.tail(3)

,id,linenb,cod_causa,cod_causa_clean,categoria_clean,testo_originale,intervallo_causa,codeonly,linecoded
178207,51509,1,D649,D649,D64,ANEMIA,NaN,NaN,1
139561,40495,3,C349,C349,C34,CA POLMONARE,NaN,NaN,1
114236,33563,1,I469,I469,I46,ARRESTO CARDIOCIRCOLATORIO,NaN,NaN,1
